In [ ]:
# ====================================================
# Library
# ====================================================
import random
from collections import defaultdict

import pandas as pd

In [ ]:
# ====================================================
# Config
# ====================================================
class CFG:
    boosted_beries = []

In [ ]:
# ====================================================
# Data Loading
# ====================================================
df_ingr = pd.read_csv("../data/m_ingredient.csv", encoding="utf-8", index_col="id")
df_berry = pd.read_csv("../data/m_berry.csv", encoding="utf-8", index_col="id")

In [ ]:
# ====================================================
# Pokemon Base Class
# ====================================================
class Pokemon:
    def __init__(
        self,
        assist_type,
        level,
        berry,
        ingrs,
        speed,
        inventory_capacity,
        main_skill,
        main_skill_level,
        sub_skills,
        personality,
        ingr_rate,
        skill_rate,
        df_berry,
        df_ingr,
        boosted_berries,
        in_good_camp=False,
        inventory_counts=0,
        energy=1
    ):
        # マスタ属性
        self.assit_type=assist_type
        self.level = level
        self.berry = berry
        self.berry_ep = 0
        self.ingrs = ingrs
        self.init_speed = speed
        self.inventory_capacity = inventory_capacity
        self.main_skill = main_skill
        self.main_skill_level = main_skill_level
        self.sub_skills = sub_skills
        self.personality = personality
        self.initial_ingr_rate = ingr_rate
        self.initial_skill_rate = skill_rate
        self.in_good_camp = in_good_camp
        # トランザクション属性
        self.speed = speed
        self.ingr_rate = ingr_rate
        self.skill_rate = skill_rate
        self.inventory_counts = inventory_counts
        self.energy = energy
        self.skill_flag = False
        self.harvested_items = defaultdict(int)
        self.ep_all = 0

        # きのみ/食材EPの更新
        # 食材ごとのEPを確認
        for i, (ingr, quantity) in enumerate(self.ingrs):
            # EP(スカラー値)を取得
            ingr_mask = df_ingr["ingredient_name"] == ingr
            ingr_ep = df_ingr.loc[ingr_mask, "ep"].values[0]
            # 食材EP属性を追加
            self.ingrs[i] = (ingr, ingr_ep, quantity)
        # きのみEP確認
        berry_mask = df_berry["berry_name"] == self.berry
        self.berry_ep = df_berry.loc[berry_mask, f"ep_{self.level}"].values[0]
        # カビゴンの好物きのみの場合はきのみ1つあたりEP2倍
        if self.berry in boosted_berries:
            berry_ep *= 2
        # 外的データは削除
        del df_berry, df_ingr, boosted_berries

        # きのみの数補正
        # とくいなもの
        self.n_berry = 2 if self.assit_type == "きのみ" else 1
        # きのみの数S
        if "きのみの数S" in self.sub_skills:
            self.n_berry += 1

        # 食材枠レベル補正
        for threshold in [30, 60]:
            if self.level < threshold:
                self.ingrs = self.ingrs[:-1]

        # サブスキル枠レベル補正
        for threshold in [10, 25, 50, 75, 100]:
            if self.level < threshold:
                self.sub_skills = self.sub_skills[:-1]

        # メインスキル発生確率補正
        # いつ育
        if self.inventory_counts == self.inventory_capacity:
            self.skill_rate = 0
        # いつ育ではない
        else:
            # 性格補正
            # 確率上昇性格(+20%)
            if self.personality in ["おだやか", "おとなしい", "しんちょう", "なまいき"]:
                self.skill_rate *= 1.2
            # 確率下降性格(-20%)
            elif self.personality in ["やんちゃ", "のうてんき", "うっかりや", "むじゃき"]:
                self.skill_rate *= 0.8

            # サブスキル補正
            # スキル確率アップM(+36%)
            if "スキル確率アップM" in self.sub_skills:
                self.skill_rate *= 1.36
            # スキル確率アップS(+18%)
            if "スキル確率アップS" in self.sub_skills:
                self.skill_rate *= 1.18

        # 食材確率補正
        # いつ育
        if self.inventory_counts == self.inventory_capacity:
            self.ingr_rate = 0
        # いつ育ではない
        else:
            # 性格補正
            # 食材確率上昇性格(+20%)
            if self.personality in ["ひかえめ", "おっとり", "うっかりや", "れいせい"]:
                self.ingr_rate *= 1.2
            # 食材確率下降性格(-20%)
            elif self.personality in ["いじっぱり", "わんぱく", "しんちょう", "ようき"]:
                self.ingr_rate *= 0.8

            # サブスキル補正
            # 食材確率アップM(+36%)
            if "食材確率アップM" in self.sub_skills:
                self.ingr_rate *= 1.36
            # 食材確率アップS(+18%)
            if "食材確率アップS" in self.sub_skills:
                self.ingr_rate *= 1.18

        # きのみ確率（1 - 食材確率）
        self.berry_rate = 1 - self.ingr_rate

        # 最大所持数補正
        if self.in_good_camp:
            self.inventory_capacity *= 1.2

        # お手スピを元気で補正
        self._update_speed()

    def activate_skill(self):
        print("スキル発動！")

    def assist(self):
        # スキル判定
        if random.random() < self.skill_rate:
            self.skill_flag = True

        # 食材お手伝い
        if random.random() < self.ingr_rate:
            # 食材をランダムでゲット
            ingr, ingr_ep, quantity = random.choice(self.ingrs)
            # EP(スカラー値)を取得
            assist_ep = ingr_ep * quantity
            self.ep_tap += assist_ep
            self.ep_all += assist_ep
            # アイテムの加算
            self.harvested_items[ingr] += quantity

            # 所持数の増加（最大所持数で高止まり）
            if self.inventory_counts + quantity < self.inventory_capacity:
                self.inventory_counts += quantity
            # 最大所持数オーバーの場合はきのみのみ（スキル抽選もなし）
            else:
                self.skill_rate = 0
                self.ingr_rate = 0
                self.inventory_counts = self.inventory_capacity

        # きのみお手伝い
        else:
            # おてつだい取得EP
            assist_ep = self.berry_ep * self.n_berry
            self.ep_tap += assist_ep
            self.ep_all += assist_ep
            # アイテムの加算
            self.harvested_items[self.berry] += self.n_berry

            # 所持数の増加（最大所持数で高止まり）
            if self.inventory_counts + self.n_berry < self.inventory_capacity:
                self.inventory_counts += self.n_berry
            # 最大所持数オーバーの場合はきのみのみ（スキル抽選もなし）
            else:
                self.skill_rate = 0
                self.ingr_rate = 0
                self.inventory_counts = self.inventory_capacity

        return assist_ep

    def idle(self, h, m, s):
        # 集めたアイテム/EPの初期化
        self.harvested_items = defaultdict(int)
        self.ep_tap = 0
        self.ep_all = 0
        init_energy = self.energy

        # 時間
        seconds = h * 3600 + m * 60 + s
        elapsed_time = 0  # 経過時間
        last_assist_time = 0  # 最後におてつだいした時刻

        while elapsed_time < seconds:
            # 次の元気消費までの時間と次のお手伝いまでの時間を計算
            time_to_next_energy_update = 600 - (elapsed_time % 600)
            time_to_next_assist = last_assist_time + self.speed - elapsed_time

            # 元気消費が先に来る場合
            if time_to_next_energy_update <= time_to_next_assist:
                # 時間切れならbreak
                if time_to_next_energy_update + elapsed_time > seconds:
                    elapsed_time = seconds
                    break
                # 経過時間の更新
                elapsed_time += time_to_next_energy_update
                # げんき0ではない場合はげんき減少
                self.energy = max(self.energy - 0.01, 0.0)
                self._update_speed()
            # お手伝いが先に来る場合
            else:
                # 時間切れならbreak
                if time_to_next_assist + elapsed_time > seconds:
                    elapsed_time = seconds
                    break
                # 経過時間の更新
                elapsed_time += time_to_next_assist
                self.assist()
                last_assist_time = elapsed_time

        # ログ
        print(f"{h}時間{m}分{s}秒 ぶんのおてつだいをしたよ！")
        print("=========")
        print("【おてつだいで集めた食材/きのみ】")
        for item, quantity in self.harvested_items.items():
            print(f"  {item:<10}：{quantity:>4}個")
        print()
        print("【おてつだいで獲得したEP】")
        print(f"  {self.ep_all} EP")
        print()
        print("【おてつだいで消費したげんき】")
        print(f"  {int(init_energy*100)}% → {int(self.energy*100)}%")

    def sleep(self, h, m, s, use_boosted_item=False):
        # げんきの初期値記憶
        init_energy = self.energy

        # 睡眠時間(分単位切り捨て)とスコア
        minute = h * 60 + m
        score = min((minute / 510) * 100, 100)

        # ログ
        print(f"{h}時間{m}分{s}秒 ぶんの睡眠計測をしたよ！(睡眠スコア: {int(score)})")
        print("寝ている間におてつだいもしたよ！")
        print("↓↓↓↓↓")
        print()

        # お手伝い
        self.idle(h, m, s)

        # げんき回復
        # 回復のおこう使用時は回復2倍
        if use_boosted_item:
            score *= 2
        self.energy = min(self.energy + (score / 100), 1)

        # ログ
        print()
        print("【睡眠計測で回復したげんき】")
        print(f"  {int(init_energy*100)}% → {int(self.energy*100)}%")

    def tap(self):
        # ログ
        print(f"タップしたよ! (今回の獲得EP: {self.ep_tap}, 総獲得EP: {self.ep_all})")

        # 総獲得EPの更新
        self.ep_all += self.ep_tap

        # 発動フラグが立っていればスキル発動
        if self.skill_flag:
            self.activate_skill()
        else:
            print("スキル発動なし...")

        # プロパティの初期化
        self.ingr_rate = self.initial_ingr_rate
        self.skill_rate = self.initial_skill_rate
        self.skill_flag = False
        self.inventory_counts = 0
        self.ep_tap = 0

    def _update_speed(self):
        # お手スピは(1 + 1.5 * げんき)倍
        self.speed = self.init_speed / (1 + 1.5 * self.energy)

        # いいキャンプチケット使用時1.2倍
        coef = 1.2 if self.in_good_camp else 1
        self.speed /= coef

In [ ]:
# ====================================================
# Raichu Example
# ====================================================
# ライチュウの場合
raichu = Pokemon(
    assist_type="きのみ",
    level=26,
    berry="ウブのみ",
    ingrs=[
        ("とくせんリンゴ", 1),
        ("とくせんリンゴ", 2),
        ("とくせんエッグ", 3),
    ],
    speed=1749,
    inventory_capacity=31,
    main_skill="エナジーチャージS",
    main_skill_level=3,
    sub_skills=[
        "きのみの数S",
        "おてつだいスピードS",
        "おてつだいスピードM",
        "スキルレベルアップS",
        "スキルレベルアップM"
    ],
    personality="さみしがり",
    ingr_rate=0.2246,
    skill_rate=0.1276,
    df_berry=df_berry,
    df_ingr=df_ingr,
    in_good_camp=False,
    boosted_berries=[],
    inventory_counts=0,
    energy=1
)

raichu.idle(24, 0, 0)